In [519]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from lib import utils
import os
from binance.spot import Spot
from dotenv import load_dotenv
load_dotenv()

True

In [520]:
bot = {
    "prices": {"updated": 0, "data": {}},
    "precisions": {"updated": 0, "data": {
        'symbol': {},
        'stepSize': {},
        'minQty': {},
        'minNotional': {},
        'thousand': {}}
    }
}

In [521]:
GATEWAY_HOST = 'https://www.binance.com'
API_PATH = '/'.join([GATEWAY_HOST, 'bapi/futures/v1'])

endpoints = {
    "positions" : {
        "path": 'public/future/copy-trade/lead-data/positions?portfolioId=%s', 
        "type": "simple",
        "params": {}
        },
    "performance" : {
        "path": 'public/future/copy-trade/lead-portfolio/performance?portfolioId=%s&timeRange=%s',
        "type": "simple",
        "params": {"timeRange": "90D"}
        },
    "detail" : {
        "path": 'friendly/future/copy-trade/lead-portfolio/detail?portfolioId=%s', 
        "type": "simple",
        "params": {}
    },
    "chart" : {
        "path": 'public/future/copy-trade/lead-portfolio/chart-data?portfolioId=%s&timeRange=%s&dataType=%s', 
        "type": "simple",
        "params": {"timeRange": "90D", "dataType": "ROI"}
    },
    "position_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/position-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    },
    "transfer_history" : {
        "path": 'public/future/copy-trade/lead-portfolio/transfer-history', 
        "type": "paginated",
        "params": {"pageNumber" : 1, "pageSize": 10}
    }
}


async def fetch_data(leaderId, endpointType, params={}):
    endpoint = endpoints[endpointType]
    # Filter out the empty params
    filtered_params = {}
    for default_key, default_value in endpoint['params'].items():
        if default_key in params.keys() and params[default_key] is not None:
            filtered_params[default_key] = params[default_key]
        else:
            filtered_params[default_key] = default_value

    if endpoint["type"] == 'simple':
        # Interpolate strings
        path = endpoint["path"] % (leaderId, *filtered_params.values())
        url = '/'.join([API_PATH, path])

        response = requests.get(url)

    if endpoint["type"] == 'paginated':
        url = '/'.join([API_PATH, endpoint["path"]])

        response = requests.post(
            url,
            json={"portfolioId": leaderId} | filtered_params,
            )

    return response.json()


async def fetch_pages(leaderId, endpointType, params={}, page_number=None, result=None, latest_item=None, reference=None, progress_bar=None):
    if page_number is None:
        page_number = 1
    if result is None:
        result = []

    response = await fetch_data(leaderId, endpointType, {"pageNumber": page_number} | params)

    if response["success"]:
        response_data = response["data"]
        response_list = response_data["list"]

        if latest_item and reference:
            response_list = sorted(response_list, key=lambda x: x[reference], reverse=True)
            item_index = 0

            for item in response_list:
                if item[reference] > latest_item[reference]:
                    result.append(item)
                    item_index += 1

                    if item_index == 10:
                        next_page = page_number + 1
                        return await fetch_pages(leaderId, endpointType, params, next_page, result, latest_item, reference)
                else:
                    # print({ "success": True, "reason": "partial", "message": f"Fetched pages {endpointType} - finished by update", "data": result })
                    return result
        else:
            result += response_list
            total_n_results = response_data["total"]
            pages_length = total_n_results // 10

            if progress_bar is None:
                progress_bar = tqdm(total=total_n_results)

            progress_bar.update(len(response_list))
            # If remainder, add another page
            if total_n_results % 10:
                pages_length += 1

            if page_number <= pages_length:
                next_page = page_number + 1

                return await fetch_pages(leaderId, endpointType, params, next_page, result, progress_bar=progress_bar)
            else:
                return result
    
    else:
        return { "success": False, "message": f"Could not fetch page {page_number}/{pages_length} of {endpointType}" }
    

In [522]:
user = {
    "updated": 0,
    "updated_date": utils.current_readable_time(),
    "auth": {
        "updated": 0,
        "data": {
            "username": "root",
            "email": "root@example.com",
            "password_hash": '',
            "binance_api_key": '',
            "binance_secret_hash": ''
            }
    },
    "detail": {
        "updated": 0,
        "data": {
            "active": True,
            "chat_id": 1031182213
            }
    },
    "account": {           
        "updated": 0,
        "data": {
            "leverage": 5,
            "value_USDT": 0,
            "value_BTC": 0,
            # "levered_ratio": np.sum(grouped_positions["ABSOLUTE_LEVERED_VALUE_SUM"]) / total_balance,
            # "unlevered_ratio": np.sum(grouped_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"]) / total_balance,
            "collateral_margin_level": 0,
            "collateral_value_USDT": 0
        }
    },
    # "leaders": {"3846188874749232129": 1, "3842534998056366337":1},
    "leaders": {
        "updated": 0,
        "data": {
            "ID" : {0: "3846188874749232129", 1:"3907342150781504256"}, 
            "WEIGHT": {0: 1, 1: 1}
        }
    },
    "positions":{
        "updated": 0,
        "data": []
    },
    "mix": {
        "updated": 0,
        "data": {'symbol': {}, 'BAG': {}}
    }
}

BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_SECRET_KEY = os.environ.get("BINANCE_SECRET_KEY")

client = Spot(api_key=BINANCE_API_KEY, api_secret=BINANCE_SECRET_KEY)

partial_update = ['auth', 'detail', 'account']

async def database_update(obj: object, update: object, collection: str) -> bool:
    current_time = utils.current_time()

    if update:
        obj["updated"] = current_time
        obj["updated_date"] = utils.current_readable_time()
        for category_key in update.keys():
            category_obj = obj[category_key]
            category_obj["updated"] = current_time
            category_data = category_obj["data"]
            
            if category_key in partial_update:
                category_data.update(update[category_key])
            else:
                obj[category_key]["data"] = update[category_key]
    
    
    # self.app.db[collection].update_one({"_id": obj["_id"]}, {"$set": update})

    return True

def aggregate_current_positions(group: pd.DataFrame) -> pd.Series:
    result = {}
    for key in group.columns:
        if key in ['leader_symbol', 'leader_markPrice_AVERAGE']:
            result[key] = group[key].iloc[0]
        else:
            result[key] = group[key].sum()
    return pd.Series(result)

def handle_current_positions(df, leaders, valueUSDT):
    df = df.merge(leaders.add_prefix("leader_"), on='leader_ID', how='inner')
    df["leader_WEIGHT_SHARE"] = df["leader_WEIGHT"] / df["leader_ID"].unique().size
    df["TARGET_SHARE"] = df["leader_WEIGHT_SHARE"] * df["leader_UNLEVERED_RATIO"] * df["leader_LEVERED_POSITION_SHARE"]
    df["TARGET_VALUE"] = valueUSDT * df["TARGET_SHARE"] * user["account"]["data"]["leverage"]
    df["TARGET_AMOUNT"] = df["TARGET_VALUE"] / df["leader_markPrice_AVERAGE"]
    df = df[["leader_symbol", "netAsset", "leader_markPrice_AVERAGE", "TARGET_VALUE", "TARGET_AMOUNT"]].groupby("leader_symbol").apply(aggregate_current_positions, include_groups=False).reset_index()
    # print(df)
    return df


def user_account_update(user, new_positions, user_leaders): #self, user
    weigth = 10

    margin_account_data = client.margin_account()

    positions = []
    for asset in margin_account_data["userAssets"]:
        amount = float(asset["netAsset"])
        if amount != 0 and asset["asset"] != 'USDT':
            positions.append(asset)

    positions = pd.DataFrame(positions)
    positions = positions.apply(lambda column: column.astype(float) if column.name != 'asset' else column)
    positions["symbol"] = positions["asset"] + 'USDT'

    assetBTC = float(margin_account_data["totalNetAssetOfBtc"])
    valueUSDT = float(client.ticker_price("BTCUSDT")["price"]) * assetBTC

    pool = positions[['symbol', 'netAsset']]
    pool = pool.merge(new_positions.add_prefix("leader_"), left_on='symbol', right_on='leader_symbol', how='outer')

    positions_closed = pool[pool["leader_symbol"].isna()]
    positions_closed["ABSOLUTE_CURRENT_VALUE"] = positions_closed["netAsset"].abs() * positions_closed["leader_markPrice_AVERAGE"]

    positions_opened = pool[(pool["symbol"].isna()) & (~pool["leader_symbol"].isna())]
    positions_opened = handle_current_positions(positions_opened, user_leaders, valueUSDT)
    positions_opened["ABSOLUTE_TARGET_VALUE"] = positions_opened["TARGET_AMOUNT"].abs() * positions_opened["leader_markPrice_AVERAGE"]

    positions_changed = pool[(~pool["symbol"].isna()) & (~pool["leader_symbol"].isna())]
    positions_changed = handle_current_positions(positions_changed, user_leaders, valueUSDT)
    positions_changed["DIFF_AMOUNT"] = positions_changed["TARGET_AMOUNT"] - positions_changed["netAsset"]
    positions_changed["CURRENT_VALUE"] = positions_changed["netAsset"] * positions_changed["leader_markPrice_AVERAGE"]
    positions_changed["ABSOLUTE_DIFF_VALUE"] = abs(positions_changed["TARGET_VALUE"] - positions_changed["CURRENT_VALUE"]) * positions_changed["leader_markPrice_AVERAGE"]
    positions_changed["OPEN"] = (positions_changed["DIFF_AMOUNT"] > 0) & (positions_changed["netAsset"] > 0) | (positions_changed["DIFF_AMOUNT"] < 0) & (positions_changed["netAsset"] < 0) | False
    positions_changed["SWITCH_DIRECTION"] = ((positions_changed["netAsset"] > 0) & (positions_changed["TARGET_AMOUNT"] < 0)) | ((positions_changed["netAsset"] < 0) & (positions_changed["TARGET_AMOUNT"] > 0))

    # pool["UNLEVERED_VALUE"] = pool["LEVERED_VALUE"] / user["account"]["data"]["leverage"]
    # pool["ABSOLUTE_LEVERED_VALUE"] = abs(pool["LEVERED_VALUE"])
    # pool["ABSOLUTE_UNLEVERED_VALUE"] = abs(pool["UNLEVERED_VALUE"])

    # levered_ratio = pool["ABSOLUTE_LEVERED_VALUE"].sum() / valueUSDT
    # unlevered_ratio = pool["ABSOLUTE_UNLEVERED_VALUE"].sum() / valueUSDT

    user_account_update = {
        "account": {
            "value_BTC": assetBTC,
            "value_USDT": valueUSDT,
            # "levered_ratio": levered_ratio,
            # "unlevered_ratio": unlevered_ratio,
            "collateral_margin_level": float(margin_account_data["totalCollateralValueInUSDT"]),
            "collateral_value_USDT": float(margin_account_data["collateralMarginLevel"])

        },
        "positions": positions.to_dict(),
    }

    return user_account_update, positions_closed[["symbol", "netAsset"]], positions_opened[["leader_symbol", "TARGET_AMOUNT", "ABSOLUTE_TARGET_VALUE"]], positions_changed[["leader_symbol", "netAsset", "TARGET_AMOUNT", "ABSOLUTE_DIFF_VALUE", "OPEN", "SWITCH_DIRECTION"]]
    # return user_account_update, positions_closed[["symbol", "netAsset"]], positions_opened[["leader_symbol", "TARGET_AMOUNT"]], positions_changed

# user_account = user_account_update(user)
# user_account_update_success = await database_update(user, user_account, 'users')
# user
# first netAsset
# "leader_positionAmount_SUM", 
# leader_markPrice_AVERAGE

In [523]:
positions_opened

,leader_symbol,TARGET_AMOUNT,ABSOLUTE_TARGET_VALUE
0,SOLUSDT,12.389773,1723.047172


In [524]:
async def leader_detail_update(leader=None, binance_id:str=None):
    if leader:
        binance_id = leader["detail"]["data"]["leadPortfolioId"]

    detail_response = await fetch_data(binance_id, 'detail')
    detail = detail_response["data"]

    detail_update = {
        "detail":  detail
    }

    return detail_update


async def leader_performance_update(leader):
    binance_id = leader["detail"]["data"]["leadPortfolioId"]

    performance_response = await fetch_data(binance_id, 'performance')
    performance = performance_response["data"]

    performance_update = {
        "performance": performance
    }

    return performance_update


async def leader_account_update(leader):
    binance_id = leader["detail"]["data"]["leadPortfolioId"]
    
    transfer_history_response = await fetch_pages(binance_id, 'transfer_history')

    transfer_history = pd.DataFrame(transfer_history_response)
    transfer_history["ID"] = binance_id
    transfer_history["INCOMING"] = transfer_history["to"] == 'Lead Trading Account'
    transfer_history["RELATIVE_VALUE"] = transfer_history.apply(lambda row: row["amount"] if row["INCOMING"] else -row["amount"], axis=1)

    position_history = await fetch_pages(binance_id, 'position_history')
    position_history = pd.DataFrame(position_history)
    position_history["ID"] = binance_id

    transfer_balance = transfer_history["RELATIVE_VALUE"].sum()
    historic_PNL = position_history["closingPnl"].sum()
    total_balance = transfer_balance + historic_PNL

    account_update = {
        "account": {
            "transfer_balance": transfer_balance,
            "historic_PNL": historic_PNL,
            "total_balance": total_balance,
        },
    }

    return account_update


sum_columns = [
    "positionAmount",
    "unrealizedProfit",
    "cumRealized",
    "notionalValue",
	"markPrice",
    "ABSOLUTE_LEVERED_VALUE",
    "ABSOLUTE_UNLEVERED_VALUE"
]

average_columns = [
    "entryPrice",
	"markPrice",
	"leverage",
	"breakEvenPrice",
]

drop_columns = [
    "id",
    "collateral",
    "isolated",
    "isolatedWallet",
    "adl",
    "askNotional",
    "bidNotional"
]


def aggregate_leader_positions(group: pd.DataFrame, handle_position_direction=False) -> pd.Series:
    result = {}
    
    for key in sum_columns: result[key] = group[key].sum() if key in group.keys() else None
    for key in average_columns: result[key] = np.average(group[key], weights=group["positionAmount"]) if key in group.keys() else None

    if handle_position_direction:
        if len(group) > 1: result["positionSide"] = "BOTH"
        else: result["positionSide"] = group["positionSide"].values[0]

    # result["LEADER_IDS"] = list(set(group["LEADER_ID"]))
    return pd.Series(result)


async def leader_positions_update(leader):
    binance_id = leader["detail"]["data"]["leadPortfolioId"]

    positions_response = await fetch_data(binance_id, 'positions')
    positions = pd.DataFrame(positions_response["data"])
    positions["ID"] = binance_id

    filtered_positions = positions.apply(lambda column: column.astype(float) if column.name in sum_columns + average_columns else column)
    filtered_positions = filtered_positions.loc[(filtered_positions["positionAmount"] != 0) | (filtered_positions["collateral"] != "USDT")]
    filtered_positions = filtered_positions.drop(columns=drop_columns)
    filtered_positions["ABSOLUTE_LEVERED_VALUE"] = abs(filtered_positions["notionalValue"])
    filtered_positions["ABSOLUTE_UNLEVERED_VALUE"] = filtered_positions["ABSOLUTE_LEVERED_VALUE"] / filtered_positions["leverage"]

    grouped_positions = filtered_positions.groupby("symbol").apply(aggregate_leader_positions, handle_position_direction=True, include_groups=False).reset_index()
    grouped_positions["ID"] = binance_id
    grouped_positions = grouped_positions.rename(columns={key: key + "_SUM" for key in sum_columns} | {key: key + "_AVERAGE" for key in average_columns})
    grouped_positions["LEVERED_POSITION_SHARE"] = grouped_positions["ABSOLUTE_LEVERED_VALUE_SUM"] / grouped_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum()
    grouped_positions["UNLEVERED_POSITION_SHARE"] = grouped_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"] / grouped_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum()

    # total_balance = leader["account"]["data"]["total_balance"]
    total_balance = float(leader["detail"]["data"]["marginBalance"])
    levered_ratio = grouped_positions["ABSOLUTE_LEVERED_VALUE_SUM"].sum() / total_balance
    unlevered_ratio = grouped_positions["ABSOLUTE_UNLEVERED_VALUE_SUM"].sum() / total_balance

    grouped_positions["LEVERED_RATIO"] = levered_ratio
    grouped_positions["UNLEVERED_RATIO"] = unlevered_ratio
    
    positions_update = {
        "account": {
           "levered_ratio": levered_ratio,
            "unlevered_ratio": unlevered_ratio,
        },
        "positions": filtered_positions.to_dict(),
        "grouped_positions": grouped_positions.to_dict()
    }

    return positions_update, grouped_positions[["symbol", "ID", "positionAmount_SUM", "markPrice_AVERAGE", "LEVERED_POSITION_SHARE", "UNLEVERED_RATIO"]]

In [525]:
roster = pd.DataFrame(columns=["ID"])
leader_mixes = pd.DataFrame()


In [564]:
async def get_symbol_precision(bot, symbol):
        symbol_precisions = pd.DataFrame(bot["precisions"]["data"]).set_index("symbol")
        thousand = False

        if symbol.startswith('1000'):
                symbol = symbol[4:]
                thousand = True
        
        if symbol in symbol_precisions.index:
                return symbol, symbol_precisions.loc[symbol]
        else:
                precision_response = client.exchange_info(symbol=symbol)

                details = precision_response['symbols'][0]
                # print(details)
                for symbol_filter in details["filters"]:
                        if symbol_filter["filterType"] == "LOT_SIZE":
                                step_size = symbol_filter["stepSize"]
                                min_quantity = float(symbol_filter["minQty"])
                        if symbol_filter["filterType"] == "NOTIONAL":
                                min_notional = float(symbol_filter['minNotional'])

                precision = [step_size, min_quantity, min_notional, thousand]

                symbol_precisions.loc[symbol] = precision

                precisions_update = {
                        "precisions": symbol_precisions
                }
                await database_update(obj=bot, update=precisions_update, collection='bot')

                return symbol, precision   



# async def get_symbol_precision(symbol:str):
#         weight = 20


#         if symbol not in bot["precisions"].keys():
#                 try:

#                 except Exception as e:
#                 if e.args[2] == 'Invalid symbol.':
#                         precision = {"stepSize": None, "minQty": None, "minNotional": None, "thousand": None}

#                 bot["precisions"][symbol] = precision
#                 await self.app.db.bot.update_one({"_id": bot["_id"]}, {"$set": {"precisions": bot["precisions"], "updatedAt": utils.current_time()}})
#         else:
#                 precision = bot["precisions"][symbol]

#         if precision["stepSize"] == None:
#                 return None, precision

#         return symbol, precision

def close_positions(closed_positions, bot):
    for symbol, position in closed_positions.set_index("symbol").iterrows():
        precision = get_symbol_precision(bot, symbol)
        print(precision)
        # if precision["stepSize"]:
            

In [565]:

async def get_leader(leader=None, binance_id:str=None):
    if leader:
        # leader = self.app.db.leaders.find_one({"_id": leader_id})
        detail = await leader_detail_update(leader=leader)

    if binance_id:
        leader = {
            "detail":{
                "data":{}
                },
            "account":{
                "data":{}
                },
            "positions":{
                "data":[]
                },
            "grouped_positions":{
                "data":[]
                },
            "mix":{
                "data":[]
                },
            "performance":{
                "data":{}
                },
        }
        detail = await leader_detail_update(binance_id=binance_id)
    
    if leader:
        await database_update(obj=leader, update=detail, collection='leaders')
        
        # performance = await leader_performance_update(leader)
        # await database_update(obj=leader, update=performance, collection='leaders')
        
        # account = await leader_account_update(leader)
        # await database_update(obj=leader, update=account, collection='leaders')

        positions, grouped_positions = await leader_positions_update(leader)
        await database_update(obj=leader, update=positions, collection='leaders')

    return leader, grouped_positions


# users = self.app.db.users.find()
users = [user]

for user in users:
    user_leaders = pd.DataFrame(user["leaders"]["data"])
    if user_leaders.size > 0:
        # leader_mixes = pd.DataFrame()

        for leader_id, leader_weight in user_leaders.set_index("ID").iterrows():
            if leader_id not in roster["ID"].unique():
                leader, leader_grouped_positions = await get_leader(binance_id=leader_id) #* in prod should be leader
                roster = pd.concat([roster, leader_grouped_positions]) if roster.size > 0 else leader_grouped_positions
            
                leader_mix = leader_grouped_positions[['symbol', 'positionAmount_SUM']].rename(columns={"positionAmount_SUM": "BAG"})
                leader_mix["BAG"] = leader_mix["BAG"] * leader_weight["WEIGHT"]
                leader_mixes = pd.concat([leader_mixes, leader_mix]) if leader_mixes.size > 0 else leader_mix
        
        user_mix = pd.DataFrame(user["mix"]["data"]).set_index("symbol")
        user_mix_new = leader_mixes.groupby('symbol').agg('sum')

        if user_mix.to_dict() != user_mix_new.to_dict():
            user_positions_new = roster[roster["ID"].isin(user_leaders["ID"].values)]#.groupby('symbol').agg(aggregate_leader_positions)
            user_account, positions_closed, positions_opened, positions_changed = user_account_update(user, user_positions_new, user_leaders)
            user_account_update_success = await database_update(obj=user, update=user_account, collection='users')

            if user_account_update_success:
                close_positions(positions_closed, bot)
                # open_positions(positions_opened)
                # change_positions(positions_changed)


# user_pool   
    # diff_user_mix(user, roster
    # if leader_id not in roster.keys():
    #     roster[leader_id] = leader
    # return roster[leader_id]

# leader_positions.head()


/var/folders/1h/861cc3q97gb3f6r361ydtk9m0000gn/T/ipykernel_76447/570481964.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positions_closed["ABSOLUTE_CURRENT_VALUE"] = positions_closed["netAsset"].abs() * positions_closed["leader_markPrice_AVERAGE"]


<coroutine object get_symbol_precision at 0x175538f40>


/var/folders/1h/861cc3q97gb3f6r361ydtk9m0000gn/T/ipykernel_76447/2980633480.py:70: RuntimeWarning: coroutine 'get_symbol_precision' was never awaited
  close_positions(positions_closed, bot)


In [570]:
roster

,symbol,ID,positionAmount_SUM,markPrice_AVERAGE,LEVERED_POSITION_SHARE,UNLEVERED_RATIO
0,ETHUSDT,3846188874749232129,31.941,2991.686098,0.833765,1.041481
1,SOLUSDT,3846188874749232129,142.000,134.169422,0.166235,1.041481
0,ETHUSDT,3907342150781504256,-5.000,3228.520000,1.000000,0.695357


In [531]:
def close_positions(closed_positions, precisions):
    for symbol, position in closed_positions.set_index("symbol").iterrows():
        if symbol not in precisions.index:
            print(symbol, position["netAsset"])

close_positions(positions_closed, symbol_precisions)

MANTAUSDT 0.00741153


In [ ]:
def get_precision(symbol):
    weight = 20
    thousand = False

    if symbol.startswith('1000'):
        symbol = symbol[4:]
        thousand = True

    if symbol not in bot["precisions"].keys():
        try:
            response = client.exchange_info(symbol=symbol)

            details = response['symbols'][0]
            # print(details)
            for symbol_filter in details["filters"]:
                if symbol_filter["filterType"] == "LOT_SIZE":
                    step_size = symbol_filter["stepSize"]
                    min_quantity = float(symbol_filter["minQty"])
                if symbol_filter["filterType"] == "NOTIONAL":
                    min_notional = float(symbol_filter['minNotional'])

            precision = {"stepSize": step_size, "minQty": min_quantity, "minNotional": min_notional, "thousand": thousand}
        except Exception as e:
            if e.args[2] == 'Invalid symbol.':
                precision = {"stepSize": None, "minQty": None, "minNotional": None, "thousand": None}

        bot["precisions"][symbol] = precision
        await self.app.db.bot.update_one({"_id": bot["_id"]}, {"$set": {"precisions": bot["precisions"], "updatedAt": utils.current_time()}})
    else:
        precision = bot["precisions"][symbol]

    if precision["stepSize"] == None:
        return None, precision
    
    return symbol, precision

In [ ]:
positions_closed

,symbol,netAsset
1,MANTAUSDT,0.007412


In [ ]:
positions_opened

,leader_symbol,TARGET_AMOUNT,ABSOLUTE_TARGET_VALUE
0,SOLUSDT,12.389773,1723.047172


In [ ]:
positions_changed

,leader_symbol,netAsset,TARGET_AMOUNT,ABSOLUTE_DIFF_VALUE,OPEN,SWITCH_DIRECTION
0,ETHUSDT,0.001499,2.786914,2.639135e+07,True,False


In [ ]:
user_mix

,symbol,BAG


In [361]:
levels = ["INFO", "ERROR"]
level = "INFO"
level_filtered = levels[levels.index(level):]
level in levels[levels.index(level)+1:]

False

In [359]:
levels[levels.index(level):]

['ERROR']

In [ ]:
ratios = [0.1, 0.5, 0.75]

for ratio in ratios:
    

In [4]:
import numpy as np

In [312]:
np.log(np.exp((1 - 0.1)) * np.log(2)) * 0.1

0.05334870794183358

In [331]:
np.exp((1 - 0.9))

1.1051709180756475

In [344]:
np.log(20) / np.log(5)

1.8613531161467862

In [352]:
np.log(1)

0.0

In [363]:
any([False, False, False])

True